In [1]:
from keras.layers import Activation,Dense,SpatialDropout1D,Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import collections
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os

In [2]:
ftrain=open(r'C:\Users\archa\OneDrive\Desktop\Downloads\LSTM-20240610T070727Z-001\LSTM\training.txt',encoding='UTF8')
ftrain

<_io.TextIOWrapper name='C:\\Users\\archa\\OneDrive\\Desktop\\Downloads\\LSTM-20240610T070727Z-001\\LSTM\\training.txt' mode='r' encoding='UTF8'>

In [3]:
max_len=0
word_freq=collections.Counter()
num_recs=0

In [4]:
for line in ftrain:
    label,sentence=line.strip().split('\t')
    words=nltk.word_tokenize(sentence.lower())
#     print(words)
    if len(words)>max_len:
        max_len=len(words)
    for word in words:
        word_freq[word]+=1
    num_recs+=1
print(max_len)
print(num_recs)
print(word_freq)

42
7086
Counter({'i': 4707, ',': 4194, '.': 3364, 'the': 3221, 'and': 2150, '!': 2131, 'potter': 2093, 'harry': 2088, 'vinci': 2001, 'da': 1998, 'brokeback': 1996, 'code': 1994, 'mountain': 1994, 'love': 1542, 'is': 1521, '...': 1441, 'a': 1305, 'was': 1179, '..': 1153, 'awesome': 1126, 'impossible': 1091, 'mission': 1090, 'like': 974, 'it': 899, 'to': 808, 'movie': 783, 'that': 718, "'s": 629, 'because': 608, 'sucks': 601, 'sucked': 596, 'hate': 578, 'so': 506, 'as': 473, 'my': 462, 'much': 452, '``': 449, 'of': 424, 'really': 374, 'movies': 366, 'stupid': 365, 'you': 330, 'down': 322, 'but': 295, 'we': 295, 'be': 290, 'with': 288, 'just': 287, 'one': 283, 'know': 276, 'suck': 276, '3': 271, 'out': 269, 'who': 263, '/': 263, 'or': 263, 'am': 259, 'loved': 256, 'want': 254, 'into': 253, 'which': 251, 'right': 249, 'for': 241, 'an': 225, "n't": 224, 'this': 213, 'me': 213, ':': 207, 'are': 203, 'think': 202, 'not': 198, 'how': 197, 'if': 195, '....': 190, 'depressing': 188, 'people': 18

In [5]:
print('unique word',len(word_freq))

unique word 2268


In [6]:
max_features=2000
max_sentence_length=40

In [7]:
vocal_size=max_features+2
vocal_size

2002

In [8]:
word_index={x[0]:i+2 for i,x in enumerate(word_freq.most_common(max_features))}
word_index

{'i': 2,
 ',': 3,
 '.': 4,
 'the': 5,
 'and': 6,
 '!': 7,
 'potter': 8,
 'harry': 9,
 'vinci': 10,
 'da': 11,
 'brokeback': 12,
 'code': 13,
 'mountain': 14,
 'love': 15,
 'is': 16,
 '...': 17,
 'a': 18,
 'was': 19,
 '..': 20,
 'awesome': 21,
 'impossible': 22,
 'mission': 23,
 'like': 24,
 'it': 25,
 'to': 26,
 'movie': 27,
 'that': 28,
 "'s": 29,
 'because': 30,
 'sucks': 31,
 'sucked': 32,
 'hate': 33,
 'so': 34,
 'as': 35,
 'my': 36,
 'much': 37,
 '``': 38,
 'of': 39,
 'really': 40,
 'movies': 41,
 'stupid': 42,
 'you': 43,
 'down': 44,
 'but': 45,
 'we': 46,
 'be': 47,
 'with': 48,
 'just': 49,
 'one': 50,
 'know': 51,
 'suck': 52,
 '3': 53,
 'out': 54,
 'who': 55,
 '/': 56,
 'or': 57,
 'am': 58,
 'loved': 59,
 'want': 60,
 'into': 61,
 'which': 62,
 'right': 63,
 'for': 64,
 'an': 65,
 "n't": 66,
 'this': 67,
 'me': 68,
 ':': 69,
 'are': 70,
 'think': 71,
 'not': 72,
 'how': 73,
 'if': 74,
 '....': 75,
 'depressing': 76,
 'people': 77,
 'his': 78,
 'would': 79,
 'horrible': 80,
 

In [9]:
word_index['PAD']=0
word_index['UNK']=1

In [10]:
print(word_index)

{'i': 2, ',': 3, '.': 4, 'the': 5, 'and': 6, '!': 7, 'potter': 8, 'harry': 9, 'vinci': 10, 'da': 11, 'brokeback': 12, 'code': 13, 'mountain': 14, 'love': 15, 'is': 16, '...': 17, 'a': 18, 'was': 19, '..': 20, 'awesome': 21, 'impossible': 22, 'mission': 23, 'like': 24, 'it': 25, 'to': 26, 'movie': 27, 'that': 28, "'s": 29, 'because': 30, 'sucks': 31, 'sucked': 32, 'hate': 33, 'so': 34, 'as': 35, 'my': 36, 'much': 37, '``': 38, 'of': 39, 'really': 40, 'movies': 41, 'stupid': 42, 'you': 43, 'down': 44, 'but': 45, 'we': 46, 'be': 47, 'with': 48, 'just': 49, 'one': 50, 'know': 51, 'suck': 52, '3': 53, 'out': 54, 'who': 55, '/': 56, 'or': 57, 'am': 58, 'loved': 59, 'want': 60, 'into': 61, 'which': 62, 'right': 63, 'for': 64, 'an': 65, "n't": 66, 'this': 67, 'me': 68, ':': 69, 'are': 70, 'think': 71, 'not': 72, 'how': 73, 'if': 74, '....': 75, 'depressing': 76, 'people': 77, 'his': 78, 'would': 79, 'horrible': 80, 'up': 81, 'reading': 82, 'why': 83, 'series': 84, 'there': 85, 'in': 86, 'only'

In [11]:
index2word={v:k for k,v in word_index.items()}
index2word

{2: 'i',
 3: ',',
 4: '.',
 5: 'the',
 6: 'and',
 7: '!',
 8: 'potter',
 9: 'harry',
 10: 'vinci',
 11: 'da',
 12: 'brokeback',
 13: 'code',
 14: 'mountain',
 15: 'love',
 16: 'is',
 17: '...',
 18: 'a',
 19: 'was',
 20: '..',
 21: 'awesome',
 22: 'impossible',
 23: 'mission',
 24: 'like',
 25: 'it',
 26: 'to',
 27: 'movie',
 28: 'that',
 29: "'s",
 30: 'because',
 31: 'sucks',
 32: 'sucked',
 33: 'hate',
 34: 'so',
 35: 'as',
 36: 'my',
 37: 'much',
 38: '``',
 39: 'of',
 40: 'really',
 41: 'movies',
 42: 'stupid',
 43: 'you',
 44: 'down',
 45: 'but',
 46: 'we',
 47: 'be',
 48: 'with',
 49: 'just',
 50: 'one',
 51: 'know',
 52: 'suck',
 53: '3',
 54: 'out',
 55: 'who',
 56: '/',
 57: 'or',
 58: 'am',
 59: 'loved',
 60: 'want',
 61: 'into',
 62: 'which',
 63: 'right',
 64: 'for',
 65: 'an',
 66: "n't",
 67: 'this',
 68: 'me',
 69: ':',
 70: 'are',
 71: 'think',
 72: 'not',
 73: 'how',
 74: 'if',
 75: '....',
 76: 'depressing',
 77: 'people',
 78: 'his',
 79: 'would',
 80: 'horrible',
 

In [12]:
x=np.empty((num_recs,),dtype=list)
x

array([None, None, None, ..., None, None, None], dtype=object)

In [13]:
x.shape

(7086,)

In [14]:
y=np.zeros(num_recs)
y

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
ftrain=open(r'C:\Users\archa\OneDrive\Desktop\Downloads\LSTM-20240610T070727Z-001\LSTM\training.txt',encoding='UTF8')
i=0
for line in ftrain:
    label,sentence=line.strip().split('\t')
    words=nltk.word_tokenize(sentence.lower())
#     print(words)
    seq=[]
    for word in words:
        if word in word_index:
            seq.append(word_index[word])
        else:
            seq.append(word_index['UNK'])
    x[i]=seq
    y[i]=int(label)
    i+=1
print(x)

[list([5, 11, 10, 13, 101, 16, 49, 21, 4])
 list([67, 19, 5, 116, 960, 961, 2, 354, 136, 111, 3, 45, 316, 315, 24, 962, 3, 6, 11, 10, 13, 137, 117, 963, 340, 67, 4])
 list([2, 123, 5, 11, 10, 13, 18, 322, 4]) ...
 list([35, 2, 300, 97, 3, 157, 5, 301, 27, 220, 3, 2, 58, 302, 39, 73, 37, 2, 303, 5, 27, 12, 14, 4])
 list([95, 12, 14, 16, 144, 18, 80, 27, 4])
 list([91, 3, 6, 12, 14, 19, 18, 89, 27, 4])]


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x=pad_sequences(x,maxlen=max_sentence_length)
x.shape

(7086, 40)

In [17]:
x

array([[  0,   0,   0, ...,  49,  21,   4],
       [  0,   0,   0, ..., 340,  67,   4],
       [  0,   0,   0, ...,  18, 322,   4],
       ...,
       [  0,   0,   0, ...,  12,  14,   4],
       [  0,   0,   0, ...,  80,  27,   4],
       [  0,   0,   0, ...,  89,  27,   4]])

In [18]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=42,test_size=0.2)

In [19]:
embedding_size=128
hidden_layer_size=64
batchsize=32
num_epochs=10

In [21]:
model=Sequential()
model.add(Embedding(vocal_size,embedding_size,input_length=40))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(hidden_layer_size,dropout=0.3,recurrent_dropout=(0.2)))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.build(input_shape=(None,40))
model.summary()

C:\Users\archa\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape          ┃      Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)       │ (None, 40, 128)       │      256,256 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ spatial_dropout1d_1           │ (None, 40, 128)       │            0 │
│ (SpatialDropout1D)            │                       │              │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ lstm_1 (LSTM)                 │ (None, 64)            │       49,408 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ dense_1 (Dense)               │ (None, 1)             │           65 │
├───────────────────────────────┼───────────────────────┼──────────────┤
│ activation_1 (Activation)     │ (None, 1)             │            0 │
└───────────────────────────────┴───────────────────────┴──────────────┘

 Total params: 305,729 (1.17 MB)

 Trainable params: 305,729 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.fit(xtrain,ytrain,batch_size=batchsize,epochs=num_epochs,validation_data=(xtest,ytest))

Epoch 1/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.7837 - loss: 0.4173 - val_accuracy: 0.9781 - val_loss: 0.0596
Epoch 2/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9903 - loss: 0.0344 - val_accuracy: 0.9873 - val_loss: 0.0349
Epoch 3/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9960 - loss: 0.0103 - val_accuracy: 0.9901 - val_loss: 0.0327
Epoch 4/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9990 - loss: 0.0051 - val_accuracy: 0.9873 - val_loss: 0.0472
Epoch 5/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9997 - loss: 0.0021 - val_accuracy: 0.9866 - val_loss: 0.0583
Epoch 6/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9996 - loss: 0.0013 - val_accuracy: 0.9817 - val_loss: 0.0523
Epoch 7/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9987 - loss: 0.0043 - val_accuracy: 0.9873 - val_loss: 0.0479
Epoch 8/10
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9997 - loss: 0.0025 - val_acc